### Depara Câmara Municipal RJ - TSE

Este notebook foi feito como um auxílio para a criação de um depara entre os nomes dos vereadores da forma que aparecem nas leis e projetos de lei da Câmara Municipal do Rio de Janeiro, e os nomes de urna dos vereadores tal qual aparecem nos dados extraídos pelo TSE.

Ele deve ser rodado após extrair as leis e projetos de lei desejados.

A função "create_depara" cria a primeira versão do depara. Em seguida, a função "correct_depara" pode ser modificada para adicionar nomes faltantes e/ou modificar e deletar nomes incorretos que estão presentes nele. Para ajudar na identificação destes nomes faltantes, a função "get_names_not_in_depara" pode ser usada para ver quais nomes de vereadores nos dados da Câmara não estão presentes no depara.

In [56]:
import re

import pandas as pd
import numpy as np
import psycopg2
from unidecode import unidecode
from sqlalchemy import create_engine

from utils import clean_author_name

In [18]:
def extract_projeto(s):
    clean_s = s.replace('.', '').replace("//", "/")
    m = re.search('(Projeto de Lei|Proj Lei)( nº)? (\d+)\-?\w? ?(/|,de|de| de|, de) ?(\d+)', clean_s)
    if m:
        nr = m.group(3).zfill(4)
        ano = m.group(5)
        return '{}/{}'.format(nr, ano)
    return ''

In [19]:
def create_depara(df_camara, df_vereadores):
    depara = {}
    for row in df_camara.iterrows():
        ano = row[1]['projeto'].split('/')[1]
        
        autores = clean_author_name(row[1]['autor'])
        autores = autores.split(',')
        autores = [autor.strip() for autor in autores]

        df_valid = df_vereadores[df_vereadores['ano_eleicao'] > int(ano) - 4]
        df_valid = df_valid[['nome_urna', 'nome', 'cpf']]

        for autor in autores:
            if autor in depara:
                continue
            
            # Alguns nomes estão exatamente iguais nos dados do TSE
            viable_names = df_valid[df_valid['nome_urna'] == autor]
            if not viable_names.empty and len(viable_names['cpf'].unique()) == 1:
                depara[autor] = (viable_names['nome_urna'].iloc[0], viable_names['cpf'].iloc[0])
                continue

            # Caso não encontre nos dados do TSE, procura por partes do nome
            viable_names = df_valid.copy()
            for part in autor.split():
                part = part.replace('.', '')
                viable1 = df_valid['nome_urna'].apply(lambda nm_dep: part in nm_dep)
                viable2 = df_valid['nome'].apply(lambda nm_dep: part in nm_dep)
                if np.any(viable1) or np.any(viable2):
                    viable_names = viable_names[viable1 | viable2]

            if not viable_names.empty and len(viable_names['cpf'].unique()) == 1:
                depara[autor] = (viable_names['nome_urna'].iloc[0], viable_names['cpf'].iloc[0])
    return depara

def get_names_not_in_depara(df_camara, depara):
    not_in_depara = set()
    for row in df_camara.iterrows():
        autores = clean_author_name(row[1]['autor'])
        autores = autores.split(',')
        autores = [autor.strip() for autor in autores]
        
        for autor in autores:
            if autor not in depara:
                not_in_depara.add(autor)
    return not_in_depara

def correct_depara(depara):
    # Adicionar nomes faltantes
    depara['DR. FERNANDO MORAES'] = ('FERNANDO MORAES', '78594200749')
    depara['PROFESSOR ROGERIO ROCAL'] = ('ROGERIO ROCAL', '04555478746')
    depara['VAL CEASA'] = ('VAL', '02867827744')
    
    # Tirar nomes errados
    depara['MARCIO GARCIA'] = ('MARCIO GARCIA', '07668281746')
    depara['ALOISIO FREITAS'] = ('ALOISIO FREITAS', '23568380749')
    
    del depara['COMISSAO DE CIENCIA TECNOLOGIA COMUNICACAO E INFORMATICA']

In [79]:
df_projetos1720 = pd.read_csv('output/projetos_lei_1720.csv', ';')
df_projetos1316 = pd.read_csv('output/projetos_lei_1316.csv', ';')
df_projetos0912 = pd.read_csv('output/projetos_lei_0912.csv', ';')
projetos = pd.concat([df_projetos1720, df_projetos1316, df_projetos0912])

projetos.dropna(subset=['ementa', 'autor'], inplace=True)
projetos.drop_duplicates(subset=['lei', 'data_publicacao'], inplace=True)

projetos['nr_lei'] = projetos['lei'].apply(lambda x: x.split('/')[0])
projetos['ano'] = projetos['lei'].apply(lambda x: x.split('/')[1])
projetos.sort_values(['ano', 'nr_lei'], ascending=False, inplace=True)
projetos.drop(['nr_lei', 'ano'], axis=1, inplace=True)

projetos = projetos.rename({'lei': 'projeto'}, axis=1)

projetos = projetos[projetos['projeto'].apply(lambda x: int(x.split('/')[1]) >= 2009)]

In [112]:
df_leis = pd.read_csv('output/leis_ordinarias.csv', ';')

df_leis['nr_projeto'] = df_leis['inteiro_teor'].apply(extract_projeto)

In [22]:
dfm = projetos.merge(
    df_leis[['lei', 'ano', 'status', 'nr_projeto']].astype(str), 
    how='left', 
    left_on='projeto', 
    right_on='nr_projeto')

dfm['status'] = dfm['status'].fillna('Não se aplica')
dfm = dfm.drop(['nr_projeto'], axis=1)

In [41]:
from sqlalchemy import create_engine
from decouple import AutoConfig
config = AutoConfig(search_path='.')

POSTGRES_USER = config('POSTGRES_USER')
POSTGRES_HOST = config('POSTGRES_HOST')
POSTGRES_PORT = config('POSTGRES_PORT')
POSTGRES_DB = config('POSTGRES_DB')

engine = create_engine(f'postgresql://{POSTGRES_USER}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

query = """
    select ano_eleicao, nome, nome_urna, cpf, sigla_partido, descricao_cargo, descricao_ue, descricao_totalizacao_turno
    from eleitoral.candidatos
    where descricao_cargo = 'VEREADOR'
    and descricao_ue = 'RIO DE JANEIRO'
    and descricao_totalizacao_turno in ('2O TURNO', 'ELEITO POR MEDIA', 'ELEITO POR QP', '2.O TURNO', 'MEDIA', 'ELEITO', 'SUPLENTE');
"""

df_tse = pd.read_sql(query, engine)

In [24]:
depara = create_depara(dfm, df_tse)
correct_depara(depara)
df_depara = pd.DataFrame(
    [[x[0], x[1][0], x[1][1]] for x in depara.items()], 
    columns=['nome_camara', 'nome_urna', 'cpf'])

/home/bernardocordeiro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [26]:
df_depara.to_sql('depara_vereadores_camara_tse', engine, schema='eleitoral', index=False)

## Experimentação

In [137]:
df_leis = pd.read_csv('output/emendas_lei_organica.csv', ';')

In [143]:
def extract_projeto_complementar(s, t=1):
    clean_s = s.replace('.', '').replace("//", "/")
    m = re.search('(ProjetodeEmendaàLeiOrgânica|PropostadeEmenda)(nº)?(\d+)\-?\w?(/|,de|de|)(\d+)', re.sub('\s', '', clean_s))
    if m:
        nr = m.group(3).zfill(4)
        ano = m.group(5)
        return '{}/{}'.format(nr, ano)
    return ''

In [144]:
df_leis['nr_projeto'] = df_leis['inteiro_teor'].apply(extract_projeto_complementar)

In [147]:
TEST = 'oi'
TEST2 = 'opa'

In [155]:
print(
    f'blabla:'
    f'{TEST}...{TEST2}'
     )

blabla:oi...opa
